<a href="https://www.kaggle.com/code/babaksh/rice-classification?scriptVersionId=223228816" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Rice Image Dataset

## **Overview**
The **Rice Image Dataset** is a collection of high-resolution images designed for rice grain classification. It contains images of five different rice varieties, making it useful for machine learning applications in agricultural research, food quality assessment, and automated classification systems.

## **Dataset Details**
- **Total Images**: 75,000  
- **Number of Classes**: 5  
- **Image Size**: 250 × 250 pixels  
- **Format**: JPEG  
- **Grayscale or RGB**: RGB  

## **Classes (Rice Varieties)**
The dataset includes images of the following five types of rice grains:  
1. **Arborio**  
2. **Basmati**  
3. **Ipsala**  
4. **Jasmine**  
5. **Karacadag**  

Each class contains an equal number of images, ensuring balanced data for training machine learning models.

## **Potential Use Cases**
- **Classification Models**: Train deep learning models (e.g., CNNs, ViTs) to classify rice varieties.
- **Feature Extraction**: Extract texture, shape, and color features for distinguishing different rice types.
- **Quality Assessment**: Identify high-quality vs. low-quality grains using automated inspection systems.
- **Agricultural Research**: Analyze grain characteristics to improve production and processing techniques.


# Code Overview

This code imports essential libraries for building and training an image classification model using TensorFlow and Keras.

- **os, numpy, pandas**: For data management and numerical operations.
- **seaborn, matplotlib**: For data visualization and plotting.
- **tensorflow**: Core framework for deep learning tasks, including model building and training.
- **sklearn**: For splitting data and dimensionality reduction (t-SNE).
- **ImageDataGenerator**: For real-time data augmentation.
- **Callbacks (EarlyStopping, ModelCheckpoint)**: To improve training by monitoring performance.

These libraries help preprocess data, define the model, optimize training, and visualize results.


In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Resizing
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Flatten
from tensorflow.keras.layers import Activation, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense

# Image Parameters and Dataset Path

In this step, we define key parameters for preprocessing and model training:

- **Image Size**: Resized to `(224, 224)` for consistency across the dataset.
- **Channels**: Set to `3` (RGB), ensuring compatibility with deep learning models.
- **Batch Size**: `64`, balancing memory usage and training efficiency.
- **Epochs**: `15`, defining the number of training iterations.
- **Learning Rate**: `1e-3`, a standard starting point for optimization.
- **Patience**: `10`, allowing early stopping if validation performance does not improve.

The dataset path is set to the **Rice Image Dataset** location on Kaggle.

In [ ]:
# Image parameters
IMG_SIZE = (224, 224)
CHANNELS = 3
BATCH_SIZE = 64
EPOCHS = 15
LEARNING_RATE = 1e-3
PATIENCE = 10

# Paths
dataset_path = "/kaggle/input/rice-image-dataset/Rice_Image_Dataset"

# Data Preparation: Train, Validation, and Test Generators

This step loads the **Rice Image Dataset** and splits it into **training, validation, and test sets** using `image_dataset_from_directory()`.

- **Training Set** (`train_generator`): 
  - 80% of the dataset.
  - Labels are inferred and one-hot encoded (`categorical` mode).
  - Images are resized to `(224, 224)`.
  - Batch size is `64`.

- **Validation Set** (`val_generator`):  
  - 20% of the dataset.
  - Used to monitor model performance during training.

- **Test Set** (`test_generator`):  
  - 10% of the dataset, separate from validation.
  - Used for final evaluation.

A **random seed (1000)** ensures reproducibility of splits.

In [ ]:
train_generator = image_dataset_from_directory(
    directory=dataset_path,
    labels='inferred',
    subset='training',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    seed=1000
)

val_generator = image_dataset_from_directory(
    directory=dataset_path,
    subset='validation',
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    seed=1000
)

test_generator = image_dataset_from_directory(
    directory=dataset_path,
    labels='inferred',
    subset='validation',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.1,
    seed=1000
)

# CNN Model Architecture

This Convolutional Neural Network (CNN) is designed for image classification and consists of three convolutional blocks followed by a classification block.

- **First Convolutional Block**: 
  - Uses a 32-filter `Conv2D` layer with a `3x3` kernel and ReLU activation.
  - Applies `BatchNormalization` to stabilize training.
  - Uses `MaxPooling2D` to reduce spatial dimensions.
  - Includes `Dropout (0.3)` to prevent overfitting.

- **Second Convolutional Block**:
  - Expands to 64 filters with similar layers as the first block.
  - `Dropout (0.4)` is used for regularization.

- **Third Convolutional Block**:
  - Uses 128 filters, `BatchNormalization`, and ReLU activation.
  - `MaxPooling2D` downsamples features.
  - `Dropout (0.5)` helps prevent overfitting.

- **Classification Block**:
  - `GlobalAveragePooling2D` followed by `Flatten` reduces feature maps.
  - Fully connected (`Dense`) layers progressively reduce dimensions: `512 → 256 → 128`.
  - `BatchNormalization` and `Dropout` (up to 0.7) improve generalization.
  - Final `Dense` layer with `softmax` activation outputs class probabilities.

This model balances depth and regularization for robust feature extraction and classification.

In [ ]:
# Build the CNN model
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), padding='same', input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.4),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.5),
        
        # Classification Block
        GlobalAveragePooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.6),
        Dense(256, activation='relu'),
        Dropout(0.65),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.7),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# AlexNet Model Architecture

This function implements a variant of the **AlexNet** architecture for image classification, modified for use with TensorFlow/Keras.

- **Input & Preprocessing**:
  - The input is resized to `227x227` to match the original AlexNet dimensions.

- **Feature Extraction (Convolutional Blocks)**:
  - **Block 1**: Uses a `96-filter` convolutional layer with an `11x11` kernel, `stride 4`, followed by `MaxPooling` and `BatchNormalization`.
  - **Block 2**: Expands to `256 filters` with a `5x5` kernel, `same padding`, followed by `MaxPooling` and `BatchNormalization`.
  - **Block 3**: Uses `384 filters` with a `3x3` kernel.
  - **Block 4**: Another `384-filter` convolutional layer with a `3x3` kernel.
  - **Block 5**: Uses `256 filters` with a `3x3` kernel, followed by `MaxPooling`.

- **Classification (Fully Connected Layers)**:
  - `GlobalAveragePooling2D` and `Flatten` prepare features for dense layers.
  - Two fully connected layers with `4096` neurons and `ReLU` activation.
  - `Dropout (0.5)` helps prevent overfitting.
  - A final `Dense` layer with `softmax` activation outputs class probabilities.

This architecture is optimized for large-scale image classification, leveraging deep convolutional layers and fully connected layers for robust feature learning.

In [ ]:
def build_alexnet_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        Input(shape=input_shape),
        tf.keras.layers.Resizing(227, 227),
        
        # Block 1
        Conv2D(96, (11, 11), strides=4, activation='relu'),
        MaxPooling2D((3, 3), strides=2),
        BatchNormalization(),
        
        # Block 2
        Conv2D(256, (5, 5), padding='same', activation='relu'),
        MaxPooling2D((3, 3), strides=2),
        BatchNormalization(),
        
        # Block 3
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        
        # Block 4
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        
        # Block 5
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        MaxPooling2D((3, 3), strides=2),
        
        # Classifier
        GlobalAveragePooling2D(),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# ResNet50 Model Architecture

This function builds a **ResNet50-based** model for image classification using transfer learning.

- **Base Model (ResNet50)**:
  - Loads a pre-trained `ResNet50` model with `ImageNet` weights.
  - `include_top=False` removes the original fully connected layers.
  - `trainable=False` freezes the base model to retain learned features.

- **Preprocessing & Feature Extraction**:
  - Input is resized to `224x224` to match ResNet50's expected dimensions.
  - Uses `tf.keras.applications.resnet50.preprocess_input()` for standardization.
  - The base model extracts deep hierarchical features.

- **Classification Head**:
  - `GlobalAveragePooling2D` reduces the feature maps to a single vector.
  - A `Dense (256 units, ReLU)` layer captures high-level representations.
  - `Dropout (0.5)` prevents overfitting.
  - The final `Dense` layer with `softmax` activation outputs class probabilities.

This approach leverages **ResNet50's** powerful feature extraction while adding a custom classification head for adaptation to new datasets.


In [ ]:
def build_resnet50_model(input_shape, num_classes):
    base_model = ResNet50(
        weights='imagenet', 
        include_top=False, 
        input_shape=(224, 224, 3)
    )
    base_model.trainable = False  

    # Build new model
    inputs = Input(shape=input_shape)
    x = Resizing(224, 224)(inputs)  # Resize to 224x224
    x = tf.keras.applications.resnet50.preprocess_input(x)
    x = base_model(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Determining Number of Classes and Input Shape

- **Number of Classes**: Extracted from `train_generator.class_names`, representing the five rice grain categories in the dataset.  
- **Class Names**: Stored in `classes`, listing all unique categories.  
- **Input Shape**: Defined as `(224, 224, 3)`, matching the resized image dimensions and RGB channels.

These values will be used to configure the model architecture for classification.


In [ ]:
# Determine the number of classes from the training generator
num_classes = len(train_generator.class_names)
classes = train_generator.class_names
input_shape = (IMG_SIZE[0], IMG_SIZE[1], CHANNELS)

# Building and Compiling CNN Models

In this step, three deep learning models are built and compiled for rice image classification:

1. **Custom CNN Model (`cnn_model`)**:
   - A sequential CNN architecture with multiple convolutional layers.
   - Uses `ReLU` activation, `BatchNormalization`, and `Dropout` for regularization.

2. **AlexNet Model (`alexnet_model`)**:
   - Based on the **AlexNet** architecture, optimized for image classification.
   - Includes large fully connected layers and `Dropout` to prevent overfitting.

3. **ResNet50 Model (`resnet50_model`)**:
   - Utilizes **ResNet50** with pre-trained `ImageNet` weights.
   - Freezes the base model for feature extraction and adds a custom classification head.

Each model is **compiled** with:
- **Optimizer**: `Adam` with a learning rate of `1e-3` for efficient training.
- **Loss Function**: `Categorical Crossentropy`, suitable for multi-class classification.
- **Evaluation Metric**: `Accuracy`, to assess model performance.

Model summaries are displayed to review the architecture and parameter count.


In [ ]:
# Build and compile the cnn model
cnn_model = build_cnn_model(input_shape, num_classes)

cnn_model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

cnn_model.summary()

In [ ]:
# Build and compile the alexnet model
alexnet_model = build_alexnet_model(input_shape, num_classes)

alexnet_model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

alexnet_model.summary()

In [ ]:
# Build and compile the resnet model
resnet50_model = build_resnet50_model(input_shape, num_classes)

resnet50_model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

resnet50_model.summary()

# Defining Callbacks for Model Training

To enhance training efficiency and prevent overfitting, two callbacks are defined:

1. **Early Stopping (`early_stopping`)**:
   - Monitors **validation loss (`val_loss`)**.
   - Stops training if no improvement is observed for `PATIENCE` epochs (set to 10).
   - Restores the best model weights to avoid overfitting.

2. **Model Checkpointing (`model_checkpoint`)**:
   - Saves the best model (`best_model.keras`) based on **validation loss**.
   - Ensures only the most optimal model is retained for further evaluation.

These callbacks help improve generalization and reduce unnecessary training time.


In [ ]:
# Define callbacks for early stopping and model checkpointing
early_stopping_cnn = EarlyStopping(
    monitor='val_loss', 
    patience=PATIENCE, 
    restore_best_weights=True,
    verbose=1
)

model_checkpoint_cnn = ModelCheckpoint(
    'best_cnn.keras', 
    monitor='val_loss', 
    save_best_only=True,
    verbose=1
)

In [ ]:
early_stopping_alexnet = EarlyStopping(
    monitor='val_loss', 
    patience=PATIENCE, 
    restore_best_weights=True,
    verbose=1
)

model_checkpoint_alexnet = ModelCheckpoint(
    'best_alexnet.keras', 
    monitor='val_loss', 
    save_best_only=True,
    verbose=1
)

In [ ]:
early_stopping_resnet = EarlyStopping(
    monitor='val_loss', 
    patience=PATIENCE, 
    restore_best_weights=True,
    verbose=1
)

model_checkpoint_resnet = ModelCheckpoint(
    'best_resnet.keras', 
    monitor='val_loss', 
    save_best_only=True,
    verbose=1
)

# Training the CNN, AlexNet, and ResNet50 Models

Each model is trained using the **fit()** function with the following settings:

- **Training Data**: `train_generator`
- **Validation Data**: `val_generator`
- **Epochs**: `20` (or until early stopping is triggered)
- **Callbacks**:
  - **Early Stopping**: Stops training if validation loss does not improve for `PATIENCE` epochs.
  - **Model Checkpoint**: Saves the best model based on validation loss.

The training process generates history objects (`cnn_history`, `alexnet_history`, and `resnet_history`), which store performance metrics like loss and accuracy for further analysis.


In [ ]:
cnn_history = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping_cnn, model_checkpoint_cnn]
)

In [ ]:
alexnet_history = alexnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping_alexnet, model_checkpoint_alexnet]
)

In [ ]:
resnet_history = resnet50_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping_resnet, model_checkpoint_resnet]
)

# Model Evaluation on Test Set

Each trained model is evaluated on the **test set** (`test_generator`) to assess its final performance:

- **CNN Model**: The test accuracy is printed for the custom CNN model.
- **AlexNet Model**: The test accuracy is printed for the AlexNet model.
- **ResNet50 Model**: The test accuracy is printed for the ResNet50 model.

These evaluations help determine the generalization capability of each model on unseen data.

In [ ]:
# Evaluate the cnn model on the test set
test_loss, test_acc = cnn_model.evaluate(test_generator)
print("Test accuracy of Simple CNN Model:", test_acc)

In [ ]:
# Evaluate the alexnet model on the test set
test_loss, test_acc = alexnet_model.evaluate(test_generator)
print("Test accuracy of Alexnet Model:", test_acc)

In [ ]:
# Evaluate the resnet model on the test set
test_loss, test_acc = resnet50_model.evaluate(test_generator)
print("Test accuracy of ResNet50 Model:", test_acc)

# Training and Test Accuracy/Loss Plots

These plots visualize the training and test accuracy, as well as the training and test loss, for the three models:

1. **Simple CNN Model**:
   - The first set of plots shows **accuracy** and **loss** for both training and test data across epochs.

2. **AlexNet Model**:
   - The second set of plots shows the **accuracy** and **loss** for the AlexNet model during training and testing.

3. **ResNet50 Model**:
   - The third set of plots displays **accuracy** and **loss** for the ResNet50 model.

These visualizations help assess model performance, check for overfitting, and compare how each model improves over epochs.


In [ ]:
# Plot training and test accuracy
plt.figure()
plt.plot(cnn_history.history['accuracy'], label='Training Accuracy')
plt.plot(cnn_history.history['val_accuracy'], label='Test Accuracy')
plt.title('Training and Test Accuracy - Simple CNN')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and test loss
plt.figure()
plt.plot(cnn_history.history['loss'], label='Training Loss')
plt.plot(cnn_history.history['val_loss'], label='Test Loss')
plt.title('Training and Test Loss -  Simple CNN')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training and test accuracy
plt.figure()
plt.plot(alexnet_history.history['accuracy'], label='Training Accuracy')
plt.plot(alexnet_history.history['val_accuracy'], label='Test Accuracy')
plt.title('Training and Test Accuracy - AlexNet')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and test loss
plt.figure()
plt.plot(alexnet_history.history['loss'], label='Training Loss')
plt.plot(alexnet_history.history['val_loss'], label='Test Loss')
plt.title('Training and Test Loss -  AlexNet')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training and test accuracy
plt.figure()
plt.plot(resnet_history.history['accuracy'], label='Training Accuracy')
plt.plot(resnet_history.history['val_accuracy'], label='Test Accuracy')
plt.title('Training and Test Accuracy - ResNet50')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and test loss
plt.figure()
plt.plot(resnet_history.history['loss'], label='Training Loss')
plt.plot(resnet_history.history['val_loss'], label='Test Loss')
plt.title('Training and Test Loss -  ResNet50')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# t-SNE Visualization for Model Feature Maps

This step visualizes the learned feature representations from each model using **t-SNE** (t-Distributed Stochastic Neighbor Embedding) to reduce high-dimensional feature maps to 2D for better understanding:

- **Feature Extraction**: 
  - Features are extracted from the global average pooling layer of each model (`gap_name` parameter).
  
- **t-SNE Transformation**: 
  - The high-dimensional features are transformed into 2D using t-SNE, making it easier to visualize and understand how the models cluster data.

- **Visualization**:
  - **Scatter plots** display the data points in 2D space, with colors corresponding to different classes.

Plots are generated for:
1. **CNN Model**: Feature maps extracted from `global_average_pooling2d`.
2. **AlexNet Model**: Feature maps from `global_average_pooling2d_1`.
3. **ResNet50 Model**: Feature maps from `global_average_pooling2d_2`.

These visualizations help assess the separability of the classes based on the learned features.


In [ ]:
cnn_model = tf.keras.models.load_model("best_cnn.keras")
alexnet_model = tf.keras.models.load_model("best_alexnet.keras")
resnet50_model = tf.keras.models.load_model("best_resnet.keras")

In [ ]:
def extract_features_and_labels(model, generator):
    features = []
    labels = []

    for batch_images, batch_labels in generator:
        features_batch = model.predict(batch_images)  # Use the model to extract features
        features.append(features_batch)
        labels.append(np.argmax(batch_labels, axis=-1))  # Convert one-hot to integer labels

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return features, labels

In [ ]:
def plot_tsne(model, gap_name, title):

    features, labels = extract_features_and_labels(model, test_generator)
    
    tsne = TSNE(n_components=2, random_state=42, perplexity=50)
    features = StandardScaler().fit_transform(features)
    features_tsne = tsne.fit_transform(features)

    plt.figure()
    sns.scatterplot(
        x=features_tsne[:, 0],
        y=features_tsne[:, 1],
        hue=labels,
        palette='viridis',
        alpha=0.7
    )
    
    plt.title(f"t-SNE Visualization for {title}")
    plt.xlabel("t-SNE Component 1")
    plt.ylabel("t-SNE Component 2")
    plt.legend(title='Class')
    plt.show()

In [ ]:
plot_tsne(cnn_model, "global_average_pooling2d", "CNN Model")

In [ ]:
plot_tsne(alexnet_model, "global_average_pooling2d_1", "AlexNet Model")

In [ ]:
plot_tsne(resnet50_model, "global_average_pooling2d_2", "ResNet50 Model")